# Create ABT

## Initial Configs

### Imports

In [2]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, expr, count, sum, max, udf, dayofweek, date_format, when, mean, median
from pyspark.sql.types import StringType
sys.path.append('../src/')
from aws.aws import Aws
import io
import pandas as pd 
import boto3
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

Python-dotenv could not parse statement starting at line 1


### Crating spark session

In [3]:
spark = SparkSession.builder.appName("ValorantDataAnalysis").getOrCreate()

spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.conf.set("spark.hadoop.fs.s3a.access.key", os.getenv('AWS_ACCESS_KEY_ID'))
spark.conf.set("spark.hadoop.fs.s3a.secret.key", os.getenv('AWS_SECRET_ACCESS_KEY'))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/30 17:41:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/30 17:41:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/30 17:41:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/05/30 17:41:51 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/05/30 17:41:51 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


### Instantiating used classes

In [4]:
aws = Aws()

## Load dataframes

### Utils

In [6]:
def get_files(bucket_name : str, folder_path : str) -> list:
    """"""
    objects = aws.list_objetcs_s3(bucket_name, folder_path)


    return objects

def concat_files_s3(objects):
    """"""

    json_files = [obj['Key'] for obj in objects]

    for file in json_files:

        response = aws.read_s3_v2(bucket_name='s3-tcc-fia-valorant', folder_path=file)
        json_data = response['Body'].read().decode('utf-8')

    return io.StringIO(json_data)

def read_spark(data_io):
    """"""
    data_io = pd.read_csv(data_io)
    return spark.createDataFrame(data_io)

def create_dataframe(bucket_name : str, folder_path : str):
    """"""
    objects = get_files(bucket_name, folder_path)
    data_io = concat_files_s3(objects)
    df = read_spark(data_io)

    return df

def save_dataframe_csv(bucket_name, folder_path, file_name, data, file_format):
    # Convert DataFrame to CSV string
    csv_buffer = io.StringIO()
    data.toPandas().to_csv(csv_buffer, index=False)

    # Retrieve CSV data from buffer
    csv_buffer_value = csv_buffer.getvalue()

    date = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = file_name + '_' + date + file_format
    file_path = folder_path + file_name

    # Write CSV string to S3
    s3 = boto3.resource('s3')

    try:
        s3.Object(bucket_name, file_path).put(Body=csv_buffer_value)
        print(f"Data was written to S3://{bucket_name}/{folder_path}")

    except Exception as e:

        print(f"Error: {e}")
    
        return False